In [ ]:
#conda install -c conda-forge mlxtend

In [1]:
#Install mlxtend and apriori
#Install mlxtend using belwo comand in conda, if it doesn't exist in spyder
#conda install -c conda-forge mlxtend 

import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
df = pd.read_excel(r"C:\Users\user\Desktop\Introtallent\Python\data\Online Retail.xlsx")

In [3]:
#check number of rows and columns
df.shape

(541909, 8)

In [4]:
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
#first transform the data
#one side customer's invoice no and all the items right side(each item as columns)
#make invoice no as row index
#quantity doesn't matter, whether the customer pruchased that or not matters so wherever quantity>=1 that is 1 otherwise 0
#altogether you need only 3 columns invoice no,description and quantity
#So after transforming data we will pas this data to our algorithm for designing the rules

In [5]:
#print unique country names from country columns
df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [ ]:
#Since buyer behavior differs from one geography to other and hence we 
#will take one country at a time for this study

In [8]:
#some of the descriptions have spaces that need to be removed
df['Description'] = df['Description'].str.strip()

In [9]:
#Check if an invoice number is missing
df.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1455
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [8]:
#drop the rows that don’t have invoice numbers
df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

In [10]:
#Looking at sales for France only for ease
basket = (df[df['Country'] =="France"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'].sum()
          .unstack() #convert vertical to horizontal 
          .reset_index().fillna(0)#when you make the data horizontal the index should be reset_index and if there is blank fill with 0
          .set_index('InvoiceNo'))

In [11]:
basket.shape #we have only 461 customers

(461, 1564)

In [11]:
basket.to_excel(r"C:\Users\user\Desktop\Introtallent\Python\final_data.xlsx")

In [13]:
# Encode -ve or 0 value transaction to 0 and +ve one to 1
def replace_quantity(x):
    if x >= 1:
        return 1
    else:
        return 0
    
# Apply Encoding
basket_sets = basket.applymap(replace_quantity) #applymap will apply the replace_quantity function in the entire basket data

In [13]:
basket_sets.to_excel(r"C:\Users\user\Desktop\Introtallent\Python\basket_data.xlsx")

In [14]:
#postage is delivery charge
#the system will think postage is most in demand product because every one is buying so support will be high
#so it starts recommending along with other products
#Delete POSTAGE item from the data. It is included in many bills toadd postage charge
basket_sets.drop('POSTAGE', inplace=True, axis=1)  

In [15]:
#generate frequent item sets that have a support of at least 7% 
#(this number was chosen so that I could get enough useful examples)
#so apriori will do is on the basket set data it will generate the support of each of the product combination
#eg support of jam,support of bread and support of bread and jam together
frequent_itemsets = apriori(basket_sets, min_support=0.07, 
                            use_colnames=True)
#after generating the support keep the items with min support 7% 
#try to keep it lower
#if you have 10 customers then keep it 1%

C:\Users\user\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:110: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [16]:
#The final step is to generate the rules with their corresponding support, confidence and lift:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules
#this will create association rule
#eg it will create support of bread with jam, support of antecedent,support of consequent,confidence of bread and jam and lift

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN CIRCUS PARADE),0.117137,0.143167,0.075922,0.648148,4.527217,0.059152,2.435209,0.882485
1,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN SPACEBOY),0.143167,0.117137,0.075922,0.530303,4.527217,0.059152,1.879645,0.909295
2,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.145336,0.143167,0.086768,0.597015,4.170059,0.065961,2.126215,0.889467
3,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.143167,0.145336,0.086768,0.606061,4.170059,0.065961,2.169531,0.887215
4,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.117137,0.145336,0.088937,0.759259,5.224157,0.071913,3.550142,0.915863
5,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN SPACEBOY),0.145336,0.117137,0.088937,0.611940,5.224157,0.071913,2.275071,0.946081
6,(SET/20 RED RETROSPOT PAPER NAPKINS),(SET/6 RED SPOTTY PAPER CUPS),0.112798,0.117137,0.086768,0.769231,6.566952,0.073555,3.825741,0.955501
7,(SET/6 RED SPOTTY PAPER CUPS),(SET/20 RED RETROSPOT PAPER NAPKINS),0.117137,0.112798,0.086768,0.740741,6.566952,0.073555,3.422064,0.960197
8,(SET/20 RED RETROSPOT PAPER NAPKINS),(SET/6 RED SPOTTY PAPER PLATES),0.112798,0.108460,0.086768,0.769231,7.092308,0.074534,3.863341,0.968215
9,(SET/6 RED SPOTTY PAPER PLATES),(SET/20 RED RETROSPOT PAPER NAPKINS),0.108460,0.112798,0.086768,0.800000,7.092308,0.074534,4.436009,0.963504


In [17]:
#We can filter the dataframe using standard pandas code. 
#In this case, look for a large lift (6) and high confidence (.8):
rules[ (rules['lift'] >= 6) & (rules['confidence'] >= 0.8) ]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
10,(SET/6 RED SPOTTY PAPER PLATES),(SET/6 RED SPOTTY PAPER CUPS),0.108460,0.117137,0.104121,0.960000,8.195556,0.091417,22.071584,0.984793
11,(SET/6 RED SPOTTY PAPER CUPS),(SET/6 RED SPOTTY PAPER PLATES),0.117137,0.108460,0.104121,0.888889,8.195556,0.091417,8.023861,0.994472
12,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER CUPS),0.086768,0.117137,0.084599,0.975000,8.323611,0.074435,35.314534,0.963457
13,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.086768,0.108460,0.084599,0.975000,8.989500,0.075188,35.661605,0.973202
14,"(SET/6 RED SPOTTY PAPER PLATES, SET/6 RED SPOT...",(SET/20 RED RETROSPOT PAPER NAPKINS),0.104121,0.112798,0.084599,0.812500,7.203125,0.072854,4.731743,0.961259


In [18]:
#export association rules to excel
rules.to_excel(r"C:\Users\user\Desktop\Introtallent\Python\output.xlsx")

* here for every item it has generated 2 set of rules by changing the andecedent and consequent
* here the algorithm says along with paper cups and napkins people buy paper plates so you can bundle them together